# 문제 6

[Kaggle 형] train_prob.csv로 문제 failure 예측하는 모델을 만들고, 

test_prob.csv에 대한 failure가 1일 확률 예측하여 다음과 같은 형식의 answer6.csv를 만들어라. 

측정 지표는 AUC(area under of ROC curve)이다. id 는 테스트 케이스의 id 이고, failure에는 failure가 1이 될 확률이다.

id,failure

16115, 0.1

16116, 0.2


In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1


In [2]:
df_train = pd.read_csv('train_prob.csv', index_col='id')
df_train.head()

,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
id,,,,,,,,,,,,,,,,,,,,,
0,A,80.10,material_7,material_8,9,5,7,8,4,18.040,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


In [3]:
df_test = pd.read_csv('test_prob.csv', index_col='id')
df_test.head()

,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
id,,,,,,,,,,,,,,,,,,,,,
16115,D,76.89,material_7,material_5,6,6,16,2,7,17.427,...,18.241,11.536,14.690,19.622,11.082,16.323,15.633,16.153,15.640,648.974
16116,D,96.13,material_7,material_5,6,6,9,10,5,18.014,...,19.056,10.746,NaN,18.475,10.174,15.710,16.274,12.229,17.204,741.103
16117,D,124.40,material_7,material_5,6,6,10,15,0,18.374,...,20.689,10.240,16.035,18.224,11.005,15.925,16.257,16.338,17.653,NaN
16118,D,175.84,material_7,material_5,6,6,4,8,15,18.056,...,19.442,12.015,18.195,19.410,10.761,16.176,16.926,13.813,13.615,NaN
16119,D,118.70,material_7,material_5,6,6,15,8,5,16.513,...,19.007,12.615,15.775,18.778,9.683,NaN,17.066,14.690,18.798,NaN


In [4]:
df_ans = pd.read_csv('test_prob_ans.csv', index_col='id')
df_ans.head()

,failure
id,
16115,1
16116,0
16117,0
16118,1
16119,0


In [5]:
df_train = df_train.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna(),
)

df_test = df_test.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna(),
)

In [6]:
df_train['product_code'].unique(), df_test['product_code'].unique()

(array(['A', 'B', 'C', 'E'], dtype=object), array(['D'], dtype=object))

In [7]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

# 위 셀의 결과를 보면, train에서 만든 모델을 test에는 적용할 수 없습니다.
# 그래서 각각을 fit_transform을 해서 결측치를 보완하면 됩니다.

# IterativeImputer 설정입니다.
imp = IterativeImputer(
    estimator=LinearRegression(),
    random_state=123
)

X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']
df_train[X_imp] = df_train.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x[X_imp]), index=x.index, columns=X_imp) 
)
df_test[X_imp] = df_test.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x[X_imp]), index=x.index, columns=X_imp) 
)

In [8]:
# 만일, product_code가 train과 test에 구분되지 않고 혼재하여 경우에 결측 보완로직을 만들어 봅니다.

def create_model():
    return IterativeImputer(
            estimator=LinearRegression(),
            random_state=123
    )

X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']

# 필요한 부분만 골라 합칩니다.
# 그 다음 product_code 별로 모델을 만들어,
# 그 모델을 Series에 담아 놓습니다.
s_imp = pd.concat([
    df_train[['product_code'] + X_imp],
    df_test[['product_code'] + X_imp],
]).groupby('product_code')[X_imp].apply(lambda x: create_model().fit(df_train[X_imp]))

# 만들어진 모델을 product_code로 구분하여 적용합니다.
df_train[X_imp] = df_train.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(s_imp[x.name].transform(x[X_imp]), index=x.index, columns=X_imp) 
)

# 만들어진 모델을 product_code로 구분하여 적용합니다.
df_test[X_imp] = df_test.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(s_imp[x.name].transform(x[X_imp]), index=x.index, columns=X_imp) 
)

In [9]:
mean_cols = ['measurement_{}'.format(i) for i in range(10, 17)]
df_train[mean_cols] = \
        df_train.groupby('product_code')[mean_cols].transform(lambda x: x.fillna(x.mean()))
df_test[mean_cols] = \
        df_test.groupby('product_code')[mean_cols].transform(lambda x: x.fillna(x.mean()))

In [10]:
df_train['loading'] = df_train['loading'].fillna(df_train['loading'].mean())
df_test['loading'] = df_test['loading'].fillna(df_train['loading'].mean())
df_train['loading_log'] = np.log(df_train['loading'])
df_test['loading_log'] = np.log(df_test['loading'])

In [11]:
# Baseline 모델을 만들고 검증하는데 필요한 기능들을 불러옵니다.
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
cv = StratifiedKFold(5, random_state=123, shuffle=True)

In [12]:
# BaseLine
# 문제 3번 전진선택법을 통해 도출한 가장 좋은 입력 변수를 사용합니다.
X_lr = ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17','na_1']

ct = ColumnTransformer([
    ('std', StandardScaler(),  ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17']),
    ('pt', 'passthrough', ['na_1'])
])
clf_lr = make_pipeline(ct, LogisticRegression(solver='lbfgs'))
scores_ = cross_val_score(clf_lr, df_train[X_lr], df_train['failure'], cv=cv, scoring='roc_auc')
score = np.mean(scores_)
scores_, score

(array([0.58840017, 0.5909264 , 0.57725538, 0.61231432, 0.58375591]),
 0.590530435171204)

In [13]:
clf_lr.fit(df_train[X_lr], df_train['failure'])
pd.DataFrame(
    clf_lr.predict_proba(df_test[X_lr])[:, 1],
    index=df_test.index,
    columns=['failure']
).to_csv('answer6.csv')

In [14]:
# 정답셋과 비교해봅니다.
roc_auc_score(df_ans['failure'], clf_lr.predict_proba(df_test[X_lr])[:, 1])

0.5883988309352517

In [15]:
# 다른 모델을 만들어 봅니다.

from sklearn.decomposition import PCA

# measurement_0~17까지 PCA하고 n_components=7를 활용

X_lr_2 = ['loading', 'na_1'] + ['measurement_{}'.format(i) for i in range(18)] 

ct_2 = ColumnTransformer([
    ('std', StandardScaler(),  ['loading']),
    ('std_pca', make_pipeline(StandardScaler(), PCA(n_components=7)), ['measurement_{}'.format(i) for i in range(18)]),
    ('pt', 'passthrough', ['na_1'])
])
clf_lr_2 = make_pipeline(ct_2, LogisticRegression(solver='lbfgs'))
scores_ = cross_val_score(clf_lr_2, df_train[X_lr_2], df_train['failure'], cv=cv, scoring='roc_auc')
score = np.mean(scores_)
scores_, score

(array([0.58690615, 0.58770217, 0.56996281, 0.61145875, 0.58306096]),
 0.5878181659505095)

In [16]:
# 정답셋과 비교해봅니다.
clf_lr_2.fit(df_train[X_lr_2], df_train['failure'])
roc_auc_score(df_ans['failure'], clf_lr_2.predict_proba(df_test[X_lr_2])[:, 1])

0.5900649730215827

In [17]:
# GradientBoosting 모델을 만들어 봅니다.
from sklearn.ensemble import GradientBoostingClassifier
X_gb = ['loading', 'na_1', 'na_2'] + ['measurement_{}'.format(i) for i in range(18)]
ct_3 = ColumnTransformer([
    ('pt', 'passthrough', X_gb)
])
clf_gb = make_pipeline(ct_3, GradientBoostingClassifier(
    n_estimators=100, max_depth=2, learning_rate=0.01, random_state=123
))
scores_ = cross_val_score(clf_gb, df_train[X_gb], df_train['failure'], cv=cv, scoring='roc_auc')
scores_, np.mean(scores_)

(array([0.59016944, 0.59366441, 0.57107023, 0.59631159, 0.57999249]),
 0.586241632091449)

In [18]:
# 문제5의 모델을 개선시켜 봅니다.
from sklearn.ensemble import RandomForestClassifier
X_rf = ['loading', 'na_1', 'na_2'] + ['measurement_{}'.format(i) for i in range(18)]
ct_4 = ColumnTransformer([
    ('std_pca', make_pipeline(StandardScaler(), PCA(n_components=7)), ['measurement_{}'.format(i) for i in range(18)]),
    ('pt', 'passthrough', ['loading', 'na_1', 'na_2'])
])
clf_rf = make_pipeline(ct_4, RandomForestClassifier(
    n_estimators=100, max_depth=7, min_samples_split= 512, random_state=123
))
scores_ = cross_val_score(clf_rf, df_train[X_rf], df_train['failure'], cv=cv, scoring='roc_auc')
scores_, np.mean(scores_)

(array([0.58317044, 0.59061429, 0.56567745, 0.59595092, 0.57646725]),
 0.5823760685674568)

In [19]:
# xgb로 모델을 만들어 봅니다.
import xgboost as xgb
X_xgb = ['loading', 'na_1', 'na_2'] + ['measurement_{}'.format(i) for i in range(18)]
ct_5 = ColumnTransformer([
    ('pt', 'passthrough', X_xgb)
])
clf_xgb = make_pipeline(ct_3, xgb.XGBClassifier(
    n_estimators=150, max_depth=2, learning_rate=0.01, random_state=123
))
scores_ = cross_val_score(clf_xgb, df_train[X_xgb], df_train['failure'], cv=cv, scoring='roc_auc')
scores_, np.mean(scores_)

(array([0.58961081, 0.59397   , 0.5726996 , 0.59593267, 0.57998353]),
 0.5864393237715635)

In [21]:
from sklearn.ensemble import VotingClassifier
X_vt = ['loading', 'na_1', 'na_2'] + ['measurement_{}'.format(i) for i in range(18)] 
# 앙상블시 가장 좋은 성능을 보인 조합입니다.
clf_vt = VotingClassifier(
    [
        ('lr', clf_lr), # 속성 선택을 통한 모델(Baseline)
        ('lr_2', clf_lr_2), # PCA + 속성 선택 모델 
        ('gb', clf_gb), # GradientBoost
        ('xgb', clf_xgb) # xgboost
    ],
    voting='soft'
)
scores_ = cross_val_score(clf_vt, df_train[X_vt], df_train['failure'], cv=cv, scoring='roc_auc')
score = np.mean(scores_)
scores_, score

(array([0.59110614, 0.59420498, 0.57448503, 0.60879559, 0.58318574]),
 0.5903554950081713)

In [22]:
# cross validation시 아주 미세하게 떨어지긴하지만,
# Voting이 일반적인 성능이 좋을 여지가 크다고 봐서 최종제출로 선택해 봅니다.

clf_vt.fit(df_train[X_vt], df_train['failure'])
pd.DataFrame(
    clf_vt.predict_proba(df_test[X_vt])[:, 1],
    index=df_test.index,
    columns=['failure']
).to_csv('answer6.csv')

In [23]:
# 정답셋과 비교해봅니다.
roc_auc_score(df_ans['failure'], clf_vt.predict_proba(df_test[X_vt])[:, 1])

0.5937891187050359